# 从零实现简化版多模态预训练模型：理解 FSDP 分布式训练

> 实验目标：构建一个简化版 CLIP 模型，在 CIFAR-10 上完成对比预训练，并深入理解 FSDP（Fully Sharded Data Parallel）分布式训练的工作原理。

## 实验目标

1. **从零构建简化版 CLIP 模型**：Vision Encoder (ViT) + Text Encoder (Transformer) + Contrastive Loss
2. **在 CIFAR-10 + 模板 Caption 上完成单 GPU 对比学习训练**，验证对比学习能否实现零样本分类
3. **深入理解 FSDP 的核心概念**：参数分片（Sharding）、梯度同步、混合精度、Checkpoint 保存
4. **掌握从单 GPU 代码迁移到 FSDP 分布式训练的完整流程**

## 预期结果

- 训练 20 个 epoch 后，Contrastive Loss 从 ~4.0 稳步降至 ~1.5
- 零样本分类准确率在 CIFAR-10 测试集上达到 ~50-70%（远超随机猜测的 10%）
- t-SNE 可视化中，同类图像嵌入和对应文本嵌入聚集在一起

## 所需环境

```
Python >= 3.9
PyTorch >= 2.0（FSDP 改进版 API）
torchvision
matplotlib
numpy
scikit-learn（用于 t-SNE 可视化）
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random
import os

# 设置随机种子，确保实验可复现
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# 检测设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name()}')
    print(f'显存: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB')

## Part 1: 数据准备

为了快速验证多模态对比学习的效果，我们使用 **CIFAR-10 数据集**配合**模板生成的 Caption**。

这种做法模拟了 CLIP 的训练方式：每张图像配有一段描述性文本。虽然我们的 Caption 是模板生成的（如 `"a photo of a cat"`），但足以验证对比学习能否让模型学会图文对齐。

**与 CLIP 原版的区别**：
- CLIP 使用 4 亿个从互联网收集的真实图文对
- 我们使用 5 万张 CIFAR-10 图像 + 8 种 Caption 模板
- 训练目标相同：让匹配的图文对在嵌入空间中靠近

In [ ]:
# ======== CIFAR-10 类别定义 ========
CIFAR10_CLASSES = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# ======== Caption 模板 ========
# 模拟 CLIP 论文中的 Prompt Engineering:
# 使用多种模板增加文本多样性，避免模型过拟合到单一模板
CAPTION_TEMPLATES = [
    'a photo of a {}',
    'a picture of a {}',
    'an image showing a {}',
    'a {} in a photograph',
    'a blurry photo of a {}',
    'a close-up of a {}',
    'a bright photo of a {}',
    'a dark photo of a {}',
]


class SimpleTokenizer:
    """简易词级别分词器

    在真实 CLIP 中使用 BPE (Byte Pair Encoding) 分词，
    这里为了教学目的使用简单的词级别分词。
    词表从所有可能的 caption 中自动构建。
    """

    def __init__(self, max_len=12):
        self.max_len = max_len
        # 从所有可能的 caption 中构建词表
        words = set()
        for cls_name in CIFAR10_CLASSES:
            for template in CAPTION_TEMPLATES:
                for word in template.format(cls_name).split():
                    words.add(word)
        # 特殊 token: <pad>=0 用于填充, <bos>=1 句首, <eos>=2 句尾
        self.word2idx = {'<pad>': 0, '<bos>': 1, '<eos>': 2}
        for i, word in enumerate(sorted(words), start=3):
            self.word2idx[word] = i
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)

    def encode(self, text):
        """将文本编码为 token id 序列"""
        tokens = [self.word2idx['<bos>']]
        tokens += [self.word2idx.get(w, 0) for w in text.split()]
        tokens += [self.word2idx['<eos>']]
        # 填充或截断到固定长度
        if len(tokens) < self.max_len:
            tokens += [0] * (self.max_len - len(tokens))
        else:
            tokens = tokens[:self.max_len]
        return tokens


class CIFAR10WithCaptions(Dataset):
    """带文本描述的 CIFAR-10 数据集

    每次取样时随机选择一个 Caption 模板，
    模拟真实训练中图文对的多样性。
    """

    def __init__(self, root='./data', train=True, tokenizer=None):
        self.dataset = datasets.CIFAR10(
            root=root, train=train, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(
                    (0.4914, 0.4822, 0.4465),
                    (0.2470, 0.2435, 0.2616)
                )
            ])
        )
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        # 随机选择一个 caption 模板，增加数据多样性
        template = random.choice(CAPTION_TEMPLATES)
        caption = template.format(CIFAR10_CLASSES[label])
        tokens = torch.tensor(self.tokenizer.encode(caption), dtype=torch.long)
        return image, tokens, label


# ======== 创建分词器和数据集 ========
tokenizer = SimpleTokenizer(max_len=12)
print(f'词表大小: {tokenizer.vocab_size}')
print(f'编码示例: "a photo of a cat" -> {tokenizer.encode("a photo of a cat")}')

train_dataset = CIFAR10WithCaptions(root='./data', train=True, tokenizer=tokenizer)
test_dataset = CIFAR10WithCaptions(root='./data', train=False, tokenizer=tokenizer)

train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True,
    num_workers=2, pin_memory=True, drop_last=True  # drop_last 保证 batch 对齐
)
test_loader = DataLoader(
    test_dataset, batch_size=256, shuffle=False,
    num_workers=2, pin_memory=True
)

print(f'训练集: {len(train_dataset)} 样本, {len(train_loader)} 个 batch')
print(f'测试集: {len(test_dataset)} 样本')

In [ ]:
# ======== 可视化几个样本 ========
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
std = torch.tensor([0.2470, 0.2435, 0.2616]).view(3, 1, 1)

for i in range(10):
    img, tokens, label = train_dataset[i * 500]
    # 反归一化
    img_display = (img * std + mean).clamp(0, 1).permute(1, 2, 0).numpy()
    # 解码 caption
    caption_words = [
        tokenizer.idx2word.get(t.item(), '')
        for t in tokens if t.item() not in (0, 1, 2)
    ]
    caption = ' '.join(caption_words)

    ax = axes[i // 5][i % 5]
    ax.imshow(img_display)
    ax.set_title(caption, fontsize=9)
    ax.axis('off')

plt.suptitle('CIFAR-10 样本及其模板 Caption', fontsize=14)
plt.tight_layout()
plt.show()

## Part 2: 模型架构 — 简化版 CLIP

我们构建一个简化版 CLIP 模型，核心架构如下：

```
图像 (32×32×3)                                文本 token 序列
     │                                              │
     ▼                                              ▼
Patch Embedding                             Token Embedding
(4×4 patch → 64 token, 256 维)              + Position Embedding
     │                                              │
     ▼                                              ▼
Vision Encoder                               Text Encoder
(6 层 Transformer)                           (4 层 Transformer)
     │                                              │
     ▼                                              ▼
  CLS Token                                 Mean Pooling
  (256 维)                                   (256 维)
     │                                              │
     ▼                                              ▼
线性投影 → L2 归一化                    线性投影 → L2 归一化
     │                                              │
     ▼                                              ▼
图像嵌入 (128 维) ──── 余弦相似度 × exp(τ) ──── 文本嵌入 (128 维)
                            │
                            ▼
                     InfoNCE Loss
```

**设计选择说明**：
- **patch_size=4**: CIFAR-10 图像仅 32×32，4×4 patch 得到 8×8=64 个 token（CLIP 原版用 14×14 patch 处理 224×224 图像）
- **embed_dim=256**: 远小于 CLIP 原版的 768/1024，适合在单 GPU 上快速训练
- **proj_dim=128**: 投影到低维共享空间，减少计算量
- **6 层视觉 / 4 层文本**: 视觉端更深，因为图像信息量远大于模板 caption

In [ ]:
class PatchEmbedding(nn.Module):
    """将图像分割为不重叠的 patch，并映射到嵌入空间

    对于 32×32 的 CIFAR-10 图像，patch_size=4 得到 8×8=64 个 patch。
    每个 patch 通过卷积层映射为 embed_dim 维向量。
    加入可学习的 CLS token 和位置编码。
    """

    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=256):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2  # 64
        # 用卷积实现 patch 切分 + 线性映射
        # 等价于: 把图像切成 patch → 展平每个 patch → 全连接层
        self.proj = nn.Conv2d(
            in_channels, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )
        # CLS token: 用于聚合全局信息（ViT 标准做法）
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim) * 0.02)
        # 可学习的绝对位置编码（CLIP 原版也是这种方式）
        self.pos_embed = nn.Parameter(
            torch.randn(1, self.num_patches + 1, embed_dim) * 0.02
        )

    def forward(self, x):
        B = x.shape[0]
        # [B, 3, 32, 32] → [B, 256, 8, 8]
        x = self.proj(x)
        # 展平空间维度: [B, 256, 8, 8] → [B, 64, 256]
        x = x.flatten(2).transpose(1, 2)
        # 拼接 CLS token: [B, 64, 256] → [B, 65, 256]
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        # 添加位置编码
        x = x + self.pos_embed
        return x


class TransformerBlock(nn.Module):
    """标准 Transformer 编码器层 (Pre-Norm 变体)

    Pre-Norm: 先 LayerNorm 再做 Attention/MLP
    这是现代 Transformer (GPT-2, ViT, LLaMA) 的标准做法，训练更稳定。

    注意：这个类同时也是 FSDP auto_wrap_policy 的包装单元。
    FSDP 会以 TransformerBlock 为粒度进行参数分片。
    """

    def __init__(self, embed_dim=256, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(
            embed_dim, num_heads, dropout=dropout, batch_first=True
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(embed_dim * mlp_ratio), embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x, key_padding_mask=None):
        # Self-Attention + 残差连接
        normed = self.norm1(x)
        x = x + self.attn(
            normed, normed, normed,
            key_padding_mask=key_padding_mask
        )[0]
        # FFN + 残差连接
        x = x + self.mlp(self.norm2(x))
        return x


class VisionEncoder(nn.Module):
    """简化版 ViT (Vision Transformer)

    与 CLIP 原版的主要区别：
    - 更小: 6 层 vs 24 层, 256 维 vs 1024 维
    - 更小的 patch: 4×4 vs 14×14（因为输入图像更小）
    - 没有 attention pooling（直接取 CLS token）
    """

    def __init__(self, img_size=32, patch_size=4, embed_dim=256,
                 num_layers=6, num_heads=8, dropout=0.1):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, 3, embed_dim)
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.patch_embed(x)  # [B, 65, 256]
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        return x[:, 0]  # 取 CLS token: [B, 256]


class TextEncoder(nn.Module):
    """简化版 Text Transformer

    与 CLIP 原版的区别：
    - 使用双向注意力（CLIP 原版用因果掩码，类似 GPT）
    - 使用 mean pooling 而非 EOS token 作为句子表征
    - 更小: 4 层 vs 12 层
    """

    def __init__(self, vocab_size, max_len=12, embed_dim=256,
                 num_layers=4, num_heads=8, dropout=0.1):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos_embed = nn.Parameter(
            torch.randn(1, max_len, embed_dim) * 0.02
        )
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # x: [B, seq_len] token indices
        padding_mask = (x == 0)  # True for <pad> positions
        x = self.token_embed(x) + self.pos_embed[:, :x.shape[1]]
        for block in self.blocks:
            x = block(x, key_padding_mask=padding_mask)
        x = self.norm(x)
        # Mean pooling: 对非 padding token 的隐藏状态取平均
        mask = (~padding_mask).unsqueeze(-1).float()  # [B, seq_len, 1]
        x = (x * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1)
        return x  # [B, 256]

In [ ]:
class MiniCLIP(nn.Module):
    """简化版 CLIP 模型

    核心组件：
    - vision_encoder: 图像编码器 (ViT)
    - text_encoder: 文本编码器 (Transformer)
    - vision_proj / text_proj: 将编码器输出投影到共享嵌入空间
    - logit_scale: 可学习的温度参数 τ（对应 CLIP 论文中的 exp(t)）
    """

    def __init__(self, vocab_size, img_size=32, patch_size=4,
                 embed_dim=256, proj_dim=128,
                 v_layers=6, t_layers=4, num_heads=8):
        super().__init__()
        self.vision_encoder = VisionEncoder(
            img_size, patch_size, embed_dim, v_layers, num_heads
        )
        self.text_encoder = TextEncoder(
            vocab_size, max_len=12, embed_dim=embed_dim,
            num_layers=t_layers, num_heads=num_heads
        )
        # 线性投影到共享空间（CLIP 原版也是线性投影，非 MLP）
        self.vision_proj = nn.Linear(embed_dim, proj_dim, bias=False)
        self.text_proj = nn.Linear(embed_dim, proj_dim, bias=False)
        # 可学习温度参数，初始化为 ln(1/0.07) ≈ 2.66
        # 训练时裁剪上界使 exp(logit_scale) ≤ 100
        self.logit_scale = nn.Parameter(torch.tensor(np.log(1 / 0.07)))

    def encode_image(self, images):
        """编码图像 → L2 归一化的嵌入向量"""
        features = self.vision_encoder(images)
        return F.normalize(self.vision_proj(features), dim=-1)

    def encode_text(self, tokens):
        """编码文本 → L2 归一化的嵌入向量"""
        features = self.text_encoder(tokens)
        return F.normalize(self.text_proj(features), dim=-1)

    def forward(self, images, tokens):
        img_embed = self.encode_image(images)   # [B, proj_dim]
        txt_embed = self.encode_text(tokens)     # [B, proj_dim]

        # 计算缩放后的余弦相似度矩阵
        logit_scale = self.logit_scale.exp().clamp(max=100.0)
        logits = logit_scale * img_embed @ txt_embed.t()  # [B, B]

        return logits, img_embed, txt_embed


def contrastive_loss(logits):
    """对称 InfoNCE 损失

    logits: [N, N] 缩放后的余弦相似度矩阵
    正确配对在对角线上：logits[i, i] 是第 i 张图像和第 i 段文本的匹配分数
    """
    N = logits.shape[0]
    labels = torch.arange(N, device=logits.device)
    # 图像→文本: 每张图像在 N 段文本中找到正确配对
    loss_i2t = F.cross_entropy(logits, labels)
    # 文本→图像: 每段文本在 N 张图像中找到正确配对
    loss_t2i = F.cross_entropy(logits.t(), labels)
    return (loss_i2t + loss_t2i) / 2


# ======== 实例化模型 ========
model = MiniCLIP(vocab_size=tokenizer.vocab_size).to(device)

# 打印参数量
total_params = sum(p.numel() for p in model.parameters())
print(f'总参数量: {total_params:,} ({total_params/1e6:.1f}M)')
print()
for name, child in model.named_children():
    n = sum(p.numel() for p in child.parameters())
    print(f'  {name}: {n:,} ({n/1e6:.2f}M)')

## Part 3: 单 GPU 训练

使用标准的 PyTorch 训练循环，训练 20 个 epoch。

**关键超参数**：
- **Batch Size = 256**: 对比学习受益于大 batch（提供更多负样本），但受限于 GPU 显存
- **学习率 = 3e-4**: 配合 AdamW 优化器和余弦退火调度
- **权重衰减 = 0.05**: 正则化，防止过拟合
- **梯度裁剪 = 1.0**: 防止训练不稳定

> 注意：这里的训练代码是标准的单 GPU 版本。在 Part 5 中，我们将展示如何将其改造为 FSDP 分布式训练。

In [ ]:
def train_one_epoch(model, dataloader, optimizer, epoch, device):
    """训练一个 epoch，返回平均 loss"""
    model.train()
    total_loss = 0
    num_batches = 0

    for batch_idx, (images, tokens, _) in enumerate(dataloader):
        images = images.to(device)
        tokens = tokens.to(device)

        # 前向传播: 计算相似度矩阵和嵌入
        logits, _, _ = model(images, tokens)
        loss = contrastive_loss(logits)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        # 梯度裁剪: 防止梯度爆炸导致训练不稳定
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

        # 每 50 个 batch 打印一次
        if batch_idx % 50 == 0:
            scale = model.logit_scale.exp().item()
            print(f'  [{batch_idx:>3d}/{len(dataloader)}] '
                  f'Loss: {loss.item():.4f}  '
                  f'Temperature: {1/scale:.4f}  '
                  f'Scale: {scale:.1f}')

    return total_loss / num_batches


# ======== 训练配置 ========
EPOCHS = 20
LR = 3e-4
WEIGHT_DECAY = 0.05

optimizer = torch.optim.AdamW(
    model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=EPOCHS, eta_min=1e-6
)

# ======== 训练循环 ========
train_losses = []
print(f'开始训练: {EPOCHS} epochs, batch_size=256, lr={LR}\n')

for epoch in range(1, EPOCHS + 1):
    loss = train_one_epoch(model, train_loader, optimizer, epoch, device)
    scheduler.step()
    train_losses.append(loss)
    current_lr = scheduler.get_last_lr()[0]
    print(f'Epoch {epoch:>2d}/{EPOCHS} | '
          f'平均 Loss: {loss:.4f} | '
          f'LR: {current_lr:.2e}\n')

print('训练完成！')

In [ ]:
# ======== 绘制训练 Loss 曲线 ========
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses,
         'b-o', linewidth=2, markersize=5)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Contrastive Loss', fontsize=12)
plt.title('训练损失曲线', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, len(train_losses) + 1))
plt.tight_layout()
plt.show()

print(f'初始 Loss: {train_losses[0]:.4f}')
print(f'最终 Loss: {train_losses[-1]:.4f}')
print(f'Loss 下降: {train_losses[0] - train_losses[-1]:.4f}')

## Part 4: 零样本分类 & 嵌入可视化

### 零样本分类

核心思路（与 CLIP 论文一致）：
1. 为每个类别生成文本描述 → `"a photo of a dog"`
2. 用 Text Encoder 编码 → 得到 10 个类别的文本嵌入
3. 对测试图像用 Image Encoder 编码 → 得到图像嵌入
4. 计算图像嵌入与所有类别文本的余弦相似度 → 选最高的作为预测

如果模型学到了有效的图文对齐，零样本准确率应远超 10%（随机猜测基线）。

In [ ]:
@torch.no_grad()
def zero_shot_eval(model, test_loader, tokenizer, device):
    """零样本分类评估"""
    model.eval()

    # 1. 编码所有类别的文本描述
    class_prompts = [f'a photo of a {cls}' for cls in CIFAR10_CLASSES]
    class_tokens = torch.stack([
        torch.tensor(tokenizer.encode(p), dtype=torch.long)
        for p in class_prompts
    ]).to(device)
    class_embeddings = model.encode_text(class_tokens)  # [10, 128]

    # 2. 对测试集逐 batch 预测
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    for images, _, labels in test_loader:
        images = images.to(device)
        image_embeddings = model.encode_image(images)  # [B, 128]

        # 计算与每个类别的余弦相似度
        similarity = image_embeddings @ class_embeddings.t()  # [B, 10]
        predictions = similarity.argmax(dim=-1).cpu()

        correct += (predictions == labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predictions.tolist())
        all_labels.extend(labels.tolist())

    accuracy = correct / total
    return accuracy, all_preds, all_labels


accuracy, preds, labels = zero_shot_eval(model, test_loader, tokenizer, device)
print(f'零样本分类准确率: {accuracy:.1%}  (随机基线: 10.0%)')

# 分类别准确率
print(f'\n各类别准确率:')
for i, cls in enumerate(CIFAR10_CLASSES):
    cls_mask = [j for j, l in enumerate(labels) if l == i]
    cls_correct = sum(1 for j in cls_mask if preds[j] == i)
    cls_acc = cls_correct / len(cls_mask) if cls_mask else 0
    print(f'  {cls:>12s}: {cls_acc:.1%}')

In [ ]:
# ======== t-SNE 嵌入空间可视化 ========
from sklearn.manifold import TSNE

@torch.no_grad()
def extract_embeddings(model, test_loader, tokenizer, device, max_samples=1000):
    """提取图像和文本嵌入用于可视化"""
    model.eval()
    img_embeds, labels_list = [], []
    count = 0

    for images, _, batch_labels in test_loader:
        if count >= max_samples:
            break
        images = images.to(device)
        img_embed = model.encode_image(images).cpu().numpy()
        img_embeds.append(img_embed)
        labels_list.append(batch_labels.numpy())
        count += images.size(0)

    # 类别文本嵌入
    class_prompts = [f'a photo of a {cls}' for cls in CIFAR10_CLASSES]
    class_tokens = torch.stack([
        torch.tensor(tokenizer.encode(p), dtype=torch.long)
        for p in class_prompts
    ]).to(device)
    txt_embed = model.encode_text(class_tokens).cpu().numpy()

    img_embeds = np.concatenate(img_embeds)[:max_samples]
    labels_arr = np.concatenate(labels_list)[:max_samples]
    return img_embeds, txt_embed, labels_arr


# 提取嵌入
img_embeds, txt_embeds, viz_labels = extract_embeddings(
    model, test_loader, tokenizer, device, max_samples=1000
)

# t-SNE 降维: 将图像嵌入和文本嵌入合并后一起降维
all_embeds = np.vstack([img_embeds, txt_embeds])
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
all_2d = tsne.fit_transform(all_embeds)

img_2d = all_2d[:len(img_embeds)]
txt_2d = all_2d[len(img_embeds):]

# 绘图
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# 图像嵌入: 小圆点
for i in range(10):
    mask = viz_labels == i
    ax.scatter(img_2d[mask, 0], img_2d[mask, 1],
               c=[colors[i]], s=10, alpha=0.4, label=CIFAR10_CLASSES[i])

# 文本嵌入: 大星号
for i in range(10):
    ax.scatter(txt_2d[i, 0], txt_2d[i, 1],
               c=[colors[i]], s=300, marker='*',
               edgecolors='black', linewidths=1, zorder=10)
    ax.annotate(CIFAR10_CLASSES[i], (txt_2d[i, 0], txt_2d[i, 1]),
                fontsize=9, fontweight='bold',
                xytext=(5, 5), textcoords='offset points')

ax.legend(loc='upper right', fontsize=8, markerscale=3)
ax.set_title('t-SNE 可视化: 图像嵌入 (圆点) + 类别文本嵌入 (星号)', fontsize=13)
ax.set_xlabel('t-SNE dim 1')
ax.set_ylabel('t-SNE dim 2')
plt.tight_layout()
plt.show()

print('如果训练有效，你应该看到:')
print('  - 同类图像聚成簇')
print('  - 每个类别的星号(文本嵌入)位于对应图像簇的中心附近')

## Part 5: FSDP 分布式训练详解

### 为什么需要 FSDP？

在实际多模态预训练中（如训练 7B+ 参数的 MLLM），单张 GPU 的显存无法容纳：
- 模型参数（FP32: ~28GB for 7B params）
- 梯度（同等大小）
- 优化器状态（Adam: 2× 参数大小，即 ~56GB）
- 总计约 **112GB**，远超单卡 80GB (A100)

**FSDP (Fully Sharded Data Parallel)** 将上述三者都分片到多张 GPU 上，每张 GPU 只持有 1/N 的模型状态。

### DDP vs FSDP 显存对比

```
DDP (Distributed Data Parallel):
┌──────────────────┐  ┌──────────────────┐
│     GPU 0        │  │     GPU 1        │
│ 完整模型参数 (P) │  │ 完整模型参数 (P) │  ← 每张 GPU 持有完整副本
│ 完整梯度     (G) │  │ 完整梯度     (G) │
│ 完整优化器   (O) │  │ 完整优化器   (O) │
│ 总计: P+G+O      │  │ 总计: P+G+O      │
└──────────────────┘  └──────────────────┘
显存占用 = N × (P + G + O)    ← 不随 GPU 数量减少！

FSDP (Fully Sharded Data Parallel):
┌──────────────────┐  ┌──────────────────┐
│     GPU 0        │  │     GPU 1        │
│ 参数分片 0  (P/N)│  │ 参数分片 1  (P/N)│  ← 每张 GPU 只有 1/N
│ 梯度分片 0  (G/N)│  │ 梯度分片 1  (G/N)│
│ 优化器分片 0(O/N)│  │ 优化器分片 1(O/N)│
│ 总计: (P+G+O)/N  │  │ 总计: (P+G+O)/N  │
└──────────────────┘  └──────────────────┘
显存占用 ≈ (P + G + O) / N   ← GPU 越多，每卡越省！
```

### FSDP 训练的通信流程

```
Forward Pass (逐层执行):
  ① All-Gather:  收集该层的完整参数 (各 GPU 的分片 → 拼接为完整参数)
  ② Compute:     用完整参数计算该层的前向传播
  ③ Discard:     丢弃非本 GPU 负责的参数分片 (释放显存)

Backward Pass (逐层执行, 反向顺序):
  ① All-Gather:  再次收集该层完整参数
  ② Compute:     计算该层的梯度
  ③ Reduce-Scatter: 将梯度归约并分片到各 GPU
  ④ Discard:     丢弃非本 GPU 负责的参数和梯度

Optimizer Step:
  - 每张 GPU 只更新自己负责的那一片参数 (无需通信)
```

### 三种分片策略

| 策略 | 对应 DeepSpeed ZeRO | 分片内容 | 显存节省 | 通信量 |
|------|---------------------|----------|----------|--------|
| `FULL_SHARD` | ZeRO Stage 3 | 参数 + 梯度 + 优化器 | **最大** | 最大 |
| `SHARD_GRAD_OP` | ZeRO Stage 2 | 梯度 + 优化器 | 中等 | 中等 |
| `NO_SHARD` | DDP | 不分片 | 无 | 最小 |

**选择建议**：
- 模型能放进单 GPU → `NO_SHARD` (就是 DDP)
- 模型参数能放下但优化器状态放不下 → `SHARD_GRAD_OP`
- 模型参数也放不下 → `FULL_SHARD`

In [ ]:
# ====================================================================
# FSDP 核心配置代码详解
#
# 注意：以下代码展示 FSDP 的关键配置步骤，不能在 notebook 中直接运行。
# FSDP 需要通过 torchrun 启动多个进程。
# 完整可运行脚本: experiments/scripts/fsdp_pretrain.py
# 运行方式: torchrun --nproc_per_node=2 experiments/scripts/fsdp_pretrain.py
# ====================================================================

print('=' * 70)
print('FSDP 关键配置步骤（教学演示）')
print('=' * 70)

# ---------- 步骤 1: 初始化分布式环境 ----------
print('''
【步骤 1】初始化分布式环境

  import torch.distributed as dist
  dist.init_process_group(backend="nccl")   # NCCL: GPU 通信最佳后端
  local_rank = int(os.environ["LOCAL_RANK"]) # torchrun 自动设置
  torch.cuda.set_device(local_rank)
''')

# ---------- 步骤 2: 配置 FSDP ----------
print('''
【步骤 2】配置 FSDP 策略

  # 2a. 自动包装策略: 以 TransformerBlock 为单位进行分片
  #     粒度太细(每个 Linear) → 通信过多
  #     粒度太粗(整个 Encoder) → 显存节省不够
  auto_wrap_policy = functools.partial(
      transformer_auto_wrap_policy,
      transformer_layer_cls={TransformerBlock},
  )

  # 2b. 混合精度: BF16 计算, 节省 ~50% 显存, 速度提升 ~2x
  #     需要 Ampere+ GPU (A100, H100)
  mp_policy = MixedPrecision(
      param_dtype=torch.bfloat16,
      reduce_dtype=torch.bfloat16,
      buffer_dtype=torch.bfloat16,
  )

  # 2c. 分片策略
  sharding = ShardingStrategy.FULL_SHARD   # ZeRO-3, 最省显存
''')

# ---------- 步骤 3: 包装模型 ----------
print('''
【步骤 3】用 FSDP 包装模型

  model = FSDP(
      model,
      sharding_strategy=sharding,
      auto_wrap_policy=auto_wrap_policy,
      mixed_precision=mp_policy,
      backward_prefetch=BackwardPrefetch.BACKWARD_PRE,  # 预取下一层
      device_id=local_rank,
  )
''')

# ---------- 步骤 4: 数据加载 ----------
print('''
【步骤 4】分布式数据加载

  # 关键改动: shuffle=True → sampler=DistributedSampler(...)
  sampler = DistributedSampler(dataset, shuffle=True)
  loader = DataLoader(dataset, batch_size=256, sampler=sampler)

  # 每个 epoch 开始时必须调用:
  sampler.set_epoch(epoch)  # 确保每轮数据顺序不同
''')

# ---------- 步骤 5: 梯度裁剪 ----------
print('''
【步骤 5】FSDP 下的梯度裁剪

  # 错误 ✗  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  # 正确 ✓  model.clip_grad_norm_(1.0)   # FSDP 提供的专用方法
''')

# ---------- 步骤 6: 保存 Checkpoint ----------
print('''
【步骤 6】FSDP Checkpoint 保存

  # 方式一: FULL_STATE_DICT (聚合到 rank 0 后保存)
  save_policy = FullStateDictConfig(offload_to_cpu=True, rank0_only=True)
  with FSDP.state_dict_type(model, StateDictType.FULL_STATE_DICT, save_policy):
      if dist.get_rank() == 0:
          torch.save(model.state_dict(), "checkpoint.pt")

  # 方式二: SHARDED_STATE_DICT (各 rank 保存自己的分片, 推荐大模型)
  # import torch.distributed.checkpoint as dcp
  # dcp.save(model.state_dict(), checkpoint_id="ckpt_dir/")
''')

# ---------- 总结: 单 GPU → FSDP 的改动清单 ----------
print('=' * 70)
print('单 GPU → FSDP 的核心改动清单')
print('=' * 70)
changes = [
    ('初始化', 'dist.init_process_group("nccl")'),
    ('模型包装', 'model = FSDP(model, ...)'),
    ('数据加载', 'shuffle=True → sampler=DistributedSampler(...)'),
    ('梯度裁剪', 'clip_grad_norm_ → model.clip_grad_norm_()'),
    ('Checkpoint', '需要 FSDP.state_dict_type 上下文管理器'),
    ('清理', '训练结束后调用 dist.destroy_process_group()'),
]
for item, detail in changes:
    print(f'  ✦ {item:12s}: {detail}')

print(f'\n完整 FSDP 训练脚本: experiments/scripts/fsdp_pretrain.py')
print(f'运行: torchrun --nproc_per_node=NUM_GPUS experiments/scripts/fsdp_pretrain.py')

## 实验结论

### 关键发现

1. **对比学习有效**: 即使在简化版模型 + 模板 Caption 的设置下，对比学习也能学到有意义的图文对齐
2. **零样本迁移**: 模型从未见过显式的类别标签，仅通过图文对比学习就获得了分类能力
3. **温度参数自适应**: `logit_scale` 在训练过程中自动调整，控制对比损失的"锐利度"

### 与 CLIP 论文的对照

| 维度 | CLIP 原版 | 本实验 |
|------|-----------|--------|
| 数据规模 | 4 亿互联网图文对 | 5 万 CIFAR-10 + 模板 Caption |
| Image Encoder | ViT-L/14 (24层, 1024维) | 简化 ViT (6层, 256维) |
| Text Encoder | 12层 GPT-2 风格 | 4层双向 Transformer |
| Batch Size | 32,768 | 256 |
| 训练规模 | 256×V100, 12天 | 单 GPU, 几分钟 |
| ImageNet Zero-Shot | 76.2% | CIFAR-10 ~50-70% |

### FSDP 学习要点

1. **何时用 FSDP**: 模型参数 × 12 (FP32 Adam) > 单卡显存时
2. **分片粒度**: 以 Transformer 层为单位包装，平衡通信与显存
3. **与 DDP 的区别**: FSDP 在 forward/backward 时动态聚合/释放参数
4. **Checkpoint**: 保存前需聚合分片参数，或使用分片 checkpoint
5. **混合精度**: BF16 训练可节省约 50% 显存，速度提升约 2x

### 后续探索方向

- 使用真实图文数据集（Flickr30k、COCO Captions）替换模板 Caption
- 增大模型规模，对比 FSDP 不同策略（FULL_SHARD vs SHARD_GRAD_OP）的显存和速度差异
- 实现 Gradient Checkpointing (`torch.utils.checkpoint`) 进一步节省显存
- 将 Vision Encoder 替换为预训练 CLIP ViT，只训练桥接层（模拟 LLaVA 的训练方式）
- 在 FSDP 脚本中添加 Wandb/TensorBoard 日志，监控分布式训练指标